In [212]:
import pickle
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
import pandas as pd
import random

# Load the saved ALS model from "../model" folder
def load_model():
    with open('../model/als_model.pkl', 'rb') as f:
        return pickle.load(f)

# Load the interaction matrix, customer_map, and product_map from "../model" folder
def load_data():
    # Load the interaction matrix and convert it to CSR format for fast row slicing
    with open('../model/interaction_matrix.pkl', 'rb') as f:
        interaction_matrix = pickle.load(f).tocsr()  # Convert to CSR format

    with open('../model/customer_map.pkl', 'rb') as f:
        customer_map = pickle.load(f)

    with open('../model/product_map.pkl', 'rb') as f:
        product_map = pickle.load(f)

    return interaction_matrix, customer_map, product_map

# Function to recommend products for a given customer ID
def recommend_products(customer_id, model, interaction_matrix, customer_map, product_map, num_recommendations=10):
    # Reverse customer_map to look up index by customer_id
    customer_index_map = {v: k for k, v in customer_map.items()}
    customer_index = customer_index_map.get(customer_id)
    print('customer_index', customer_index)
    
    if customer_index is None:
        return "Customer not found"

    # Extract the user-specific interaction history from the matrix
    user_interaction = interaction_matrix[customer_index]

    # Get recommendations using the ALS model
    recommendations = model.recommend(customer_index, user_interaction, N=num_recommendations)
    print(recommendations)

    # Filter out invalid recommendations and map back the recommended product indices to product IDs
    recommended_product_ids = []
    for product_idx in recommendations[0]:
        # Ensure the product index is valid and exists in product_map
        if product_idx in product_map:
            recommended_product_ids.append(product_map[product_idx])
        else:
            print(f"Invalid product index: {product_idx}")

    return recommended_product_ids if recommended_product_ids else "No valid recommendations"

# Example usage
als_model = load_model()
interaction_matrix, customer_map, product_map = load_data()


In [213]:
customers = pd.read_csv('../data/transactions.csv').customer_id.unique().tolist()